# Hotel Booking Demand Analysis 

<img src="https://images.unsplash.com/photo-1569959595862-1c84553361c1?ixid=MXwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHw%3D&ixlib=rb-1.2.1&auto=format&fit=crop&w=2934&q=80" width="800">

Our analysis is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
  3. Identify **data entities**, **metrics** and **dimensions**
  4. **Columns/fields categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. **Arrival time related** columns basic profiling
  2. **Guest info related** columns basic profiling
  3. **Order related** columns basic profiling
  3. **Issue related** columns basic profiling
5. **Answer some business questions** to improve service
  1. **Create new columns** based on the profiling we saw earlier
  2. **Comparison across dimension** compare cancelation rate across dimension 

Let's go for it:

## 1. PySpark environment setup

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## 2. Data source and Spark DataFrame setup

In [5]:
booking_df = ( spark.read
                    .option("inferSchema", "true")
                    .option("header","true")
                    .csv("hotel_bookings.csv")
                    .cache()
             )

In [6]:
booking_df.show(1)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_parking_spaces|total_of

## 3. Dataset metadata analysis

### A. Display schema and size of the DataFrame

In [7]:
from IPython.display import display, Markdown

In [8]:
booking_df.printSchema()
display(Markdown("This dataframe has **%d rows**." % booking_df.count()))

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

This dataframe has **119390 rows**.

We could see here that `arrival_date_month` and `children` are in string but should have been in integer, based on their defintions and columns similar to them. We'll keep this mind for our inspection later.

### B. Get one or multiple random samples from the data set

In [9]:
display(booking_df.take(1))

[Row(hotel='Resort Hotel', is_canceled=0, lead_time=342, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Direct', distribution_channel='Direct', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='C', assigned_room_type='C', booking_changes=3, deposit_type='No Deposit', agent='NULL', company='NULL', days_in_waiting_list=0, customer_type='Transient', adr=0.0, required_car_parking_spaces=0, total_of_special_requests=0, reservation_status='Check-Out', reservation_status_date='2015-07-01')]

In [10]:
display(booking_df.sample(0.1).take(1))

[Row(hotel='Resort Hotel', is_canceled=0, lead_time=72, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=2, stays_in_week_nights=4, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Direct', distribution_channel='Direct', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='D', assigned_room_type='D', booking_changes=1, deposit_type='No Deposit', agent='250', company='NULL', days_in_waiting_list=0, customer_type='Transient', adr=99.67, required_car_parking_spaces=0, total_of_special_requests=1, reservation_status='Check-Out', reservation_status_date='2015-07-07')]

In [11]:
booking_df.show(1)

+------------+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|       hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_parking_spaces|total_of

### C. Checking for NULLs across the data set

In [12]:
print("Checking for nulls in the dataset:")

booking_df_column_names = booking_df.columns
booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in booking_df_column_names]).show()

Checking for nulls in the dataset:
+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_pa

We could see that in the first row & the sampled row there are "Null"s, but they are not considered Null in Spark. 

For Spark, null values need to be named `None` instead of `NULL`, so we'll replace these cells with `None`. 

In [13]:
booking_df_column_names = booking_df.columns

for i in booking_df_column_names:
    booking_df = booking_df.withColumn(i,when((col(i)=='NULL'),None).otherwise(col(i)))

In [14]:
print("Checking for nulls in the dataset:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in booking_df_column_names]).show()

Checking for nulls in the dataset:
+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_pa

### D. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** hotel
* **Metrics:** lead_time, booking_changes, adults...
* **Dimensions:** country, market segment, distribution channel...

### E. Column categorization

The following could be a potential column categorization:

* **Arrival time related columns:** *arrival_date_year*, *arrival_date_month*, *arrival_date_week_number*, *arrival_date_day_of_month*, *reservation_status_date*
* **Issue related columns:** *is_cancelled*, *previous_cancellation*, *booking_changes*, *reservation_status*
* **Guest info related columns:** *country*, *adults*, *children*, *babies*, *is_repeated_guest*, *customer_type*, *previous_bookings_not_canceled*
* **Order related columns:** *market_segment*, *distribution_channel*, *reserved_room_type*, *deposit_type*, *agent*, *reservation_status*, *lead_time*, *agent* ...

## 4. Columns groups basic profiling to better understand our data set
### A. Arrival time related columns basic profiling

In [15]:
arrival_time_related_columns = ["arrival_date_year", "arrival_date_month", "arrival_date_week_number", 
                                "arrival_date_day_of_month", "reservation_status_date"]

In [16]:
print("Summary of arrival time related columns:")

booking_df.select(arrival_time_related_columns).summary().show()

Summary of arrival time related columns:
+-------+------------------+------------------+------------------------+-------------------------+-----------------------+
|summary| arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|reservation_status_date|
+-------+------------------+------------------+------------------------+-------------------------+-----------------------+
|  count|            119390|            119390|                  119390|                   119390|                 119390|
|   mean| 2016.156554150264|              null|       27.16517296255968|       15.798241058715135|                   null|
| stddev|0.7074759445220408|              null|      13.605138355497665|        8.780829470578343|                   null|
|    min|              2015|             April|                       1|                        1|             2014-10-17|
|    25%|              2016|              null|                      16|                        8|

We could see that `reservation_status_date` in string format, we will convert this column into datetime format.

In [17]:
booking_df = booking_df.withColumn("reservation_status_date", 
                      to_date(unix_timestamp(col("reservation_status_date"), "yyyy-MM-dd").cast("timestamp")))

In [18]:
print("Checking for nulls in the time related columns:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in arrival_time_related_columns]).show()

Checking for nulls in the time related columns:
+-----------------+------------------+------------------------+-------------------------+-----------------------+
|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|reservation_status_date|
+-----------------+------------------+------------------------+-------------------------+-----------------------+
|                0|                 0|                       0|                        0|                      0|
+-----------------+------------------+------------------------+-------------------------+-----------------------+



We realized that the `arrival_date_month` is in text instead of number, which may cause trouble is we want to do calculation later on, so we will transfrom this column.

In [19]:
monthDict = {"January": "1", "February": "2", "March": "3", "April": "4", 
             "May": "5", "June": "6", "July":"7", "August": "8", 
             "September": "9", "October": "10", "November": "11", "December": "12"}
booking_df = booking_df.na.replace(monthDict, 1)
booking_df = booking_df.withColumn("arrival_date_month", booking_df["arrival_date_month"].cast(IntegerType()))

/opt/spark-3.0.0-bin-hadoop3.2/python/pyspark/sql/dataframe.py:1894: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [20]:
print("Summary of arrival time related columns:")

booking_df.select(arrival_time_related_columns).summary().show()

Summary of arrival time related columns:
+-------+------------------+------------------+------------------------+-------------------------+
|summary| arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|
+-------+------------------+------------------+------------------------+-------------------------+
|  count|            119390|            119390|                  119390|                   119390|
|   mean| 2016.156554150264| 6.552483457576011|       27.16517296255968|       15.798241058715135|
| stddev|0.7074759445220408| 3.090618686900272|      13.605138355497665|        8.780829470578343|
|    min|              2015|                 1|                       1|                        1|
|    25%|              2016|                 4|                      16|                        8|
|    50%|              2016|                 7|                      28|                       16|
|    75%|              2017|                 9|                     

In [21]:
print("Checking for unique values the time related columns:")

booking_df.select([countDistinct(c).alias(c) for c in arrival_time_related_columns]).show()

Checking for unique values the time related columns:
+-----------------+------------------+------------------------+-------------------------+-----------------------+
|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|reservation_status_date|
+-----------------+------------------+------------------------+-------------------------+-----------------------+
|                3|                12|                      53|                       31|                    926|
+-----------------+------------------+------------------------+-------------------------+-----------------------+



In [22]:
print ("Most and least frequent occurrences for arrival_date_month:")
monthOccurrenceDF = booking_df.groupBy("arrival_date_month").count().sort(col("count").desc())
monthOccurrenceDF.show()

Most and least frequent occurrences for arrival_date_month:
+------------------+-----+
|arrival_date_month|count|
+------------------+-----+
|                 8|13877|
|                 7|12661|
|                 5|11791|
|                10|11160|
|                 4|11089|
|                 6|10939|
|                 9|10508|
|                 3| 9794|
|                 2| 8068|
|                11| 6794|
|                12| 6780|
|                 1| 5929|
+------------------+-----+



In [23]:
print ("Most and least frequent occurrences for arrival_date_day_of_month:")
dayOccurrenceDF = booking_df.groupBy("arrival_date_day_of_month").count().sort(col("count").desc())
dayOccurrenceDF.show()

Most and least frequent occurrences for arrival_date_day_of_month:
+-------------------------+-----+
|arrival_date_day_of_month|count|
+-------------------------+-----+
|                       17| 4406|
|                        5| 4317|
|                       15| 4196|
|                       25| 4160|
|                       26| 4147|
|                        9| 4096|
|                       12| 4087|
|                       16| 4078|
|                        2| 4055|
|                       19| 4052|
|                       20| 4032|
|                       18| 4002|
|                       24| 3993|
|                       28| 3946|
|                        8| 3921|
|                        3| 3855|
|                       30| 3853|
|                        6| 3833|
|                       14| 3819|
|                       27| 3802|
+-------------------------+-----+
only showing top 20 rows



In [24]:
print ("Most and least frequent occurrences for arrival_date_week_number:")
weekOccurrenceDF = booking_df.groupBy("arrival_date_week_number").count().sort(col("count").desc())
weekOccurrenceDF.show(53)

Most and least frequent occurrences for arrival_date_week_number:
+------------------------+-----+
|arrival_date_week_number|count|
+------------------------+-----+
|                      33| 3580|
|                      30| 3087|
|                      32| 3045|
|                      34| 3040|
|                      18| 2926|
|                      21| 2854|
|                      28| 2853|
|                      17| 2805|
|                      20| 2785|
|                      29| 2763|
|                      42| 2756|
|                      31| 2741|
|                      41| 2699|
|                      15| 2689|
|                      27| 2664|
|                      25| 2663|
|                      38| 2661|
|                      23| 2621|
|                      35| 2593|
|                      39| 2581|
|                      22| 2546|
|                      24| 2498|
|                      13| 2416|
|                      16| 2405|
|                      19| 2402|
|         

In [25]:
print ("Most and least frequent occurrences for reservation_status_date:")
reservationOccurrenceDF = booking_df.groupBy("reservation_status_date").count().sort(col("count").desc())
reservationOccurrenceDF.show()

Most and least frequent occurrences for reservation_status_date:
+-----------------------+-----+
|reservation_status_date|count|
+-----------------------+-----+
|             2015-10-21| 1461|
|             2015-07-06|  805|
|             2016-11-25|  790|
|             2015-01-01|  763|
|             2016-01-18|  625|
|             2015-07-02|  469|
|             2016-12-07|  450|
|             2015-12-18|  423|
|             2016-02-09|  412|
|             2016-04-04|  382|
|             2017-01-24|  343|
|             2016-11-21|  340|
|             2016-03-15|  329|
|             2017-01-19|  321|
|             2017-02-02|  315|
|             2016-09-20|  303|
|             2016-04-17|  299|
|             2017-05-05|  297|
|             2015-09-09|  290|
|             2016-04-27|  283|
+-----------------------+-----+
only showing top 20 rows



In [26]:
reservationOccurrenceDF.sort(col("count")).show()

+-----------------------+-----+
|reservation_status_date|count|
+-----------------------+-----+
|             2015-03-10|    1|
|             2015-04-30|    1|
|             2015-04-21|    1|
|             2015-02-26|    1|
|             2015-02-06|    1|
|             2015-03-18|    1|
|             2017-09-12|    1|
|             2015-03-05|    1|
|             2015-02-27|    1|
|             2015-02-19|    1|
|             2015-03-28|    1|
|             2015-02-09|    1|
|             2015-03-11|    1|
|             2015-04-25|    1|
|             2015-02-24|    1|
|             2015-03-13|    1|
|             2015-01-29|    1|
|             2015-03-12|    1|
|             2015-04-27|    1|
|             2015-04-07|    1|
+-----------------------+-----+
only showing top 20 rows



### B. Guest info related columns basic profiling

In [27]:
guest_info_related_columns = ["country", "adults", "children", "babies", "is_repeated_guest", 
                              "previous_bookings_not_canceled", "customer_type"]

In [28]:
print("Summary of guest info related columns:")

booking_df.select(guest_info_related_columns).summary().show()

Summary of guest info related columns:
+-------+-------+------------------+-------------------+--------------------+-------------------+------------------------------+---------------+
|summary|country|            adults|           children|              babies|  is_repeated_guest|previous_bookings_not_canceled|  customer_type|
+-------+-------+------------------+-------------------+--------------------+-------------------+------------------------------+---------------+
|  count| 118902|            119390|             119390|              119390|             119390|                        119390|         119390|
|   mean|   null|1.8564033838679956|0.10388990333874994|0.007948739425412514|0.03191222045397437|           0.13709690928888515|           null|
| stddev|   null|0.5792609988327531| 0.3985614447864427|  0.0974361913012642|0.17576714541065672|             1.497436847707679|           null|
|    min|    ABW|                 0|                  0|                   0|              

In [29]:
print("Checking for nulls in the guest related columns:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in guest_info_related_columns]).show()

Checking for nulls in the guest related columns:
+-------+------+--------+------+-----------------+------------------------------+-------------+
|country|adults|children|babies|is_repeated_guest|previous_bookings_not_canceled|customer_type|
+-------+------+--------+------+-----------------+------------------------------+-------------+
|    488|     0|       0|     0|                0|                             0|            0|
+-------+------+--------+------+-----------------+------------------------------+-------------+



We could see there are 488 null values in the dataframe, let's replace it by `Other countries`. 

In [30]:
booking_df = booking_df.fillna("Other countries", "country")

In [31]:
print("Checking for unique values the guest info related columns:")

booking_df.select([countDistinct(c).alias(c) for c in guest_info_related_columns]).show()

Checking for unique values the guest info related columns:
+-------+------+--------+------+-----------------+------------------------------+-------------+
|country|adults|children|babies|is_repeated_guest|previous_bookings_not_canceled|customer_type|
+-------+------+--------+------+-----------------+------------------------------+-------------+
|    178|    14|       6|     5|                2|                            73|            4|
+-------+------+--------+------+-----------------+------------------------------+-------------+



In [32]:
print ("Most and least frequent occurrences for country:")
countryOccurrenceDF = booking_df.groupBy("country").count().sort(col("count").desc())
countryOccurrenceDF.show(30)

Most and least frequent occurrences for country:
+---------------+-----+
|        country|count|
+---------------+-----+
|            PRT|48590|
|            GBR|12129|
|            FRA|10415|
|            ESP| 8568|
|            DEU| 7287|
|            ITA| 3766|
|            IRL| 3375|
|            BEL| 2342|
|            BRA| 2224|
|            NLD| 2104|
|            USA| 2097|
|            CHE| 1730|
|             CN| 1279|
|            AUT| 1263|
|            SWE| 1024|
|            CHN|  999|
|            POL|  919|
|            ISR|  669|
|            RUS|  632|
|            NOR|  607|
|            ROU|  500|
|Other countries|  488|
|            FIN|  447|
|            DNK|  435|
|            AUS|  426|
|            AGO|  362|
|            LUX|  287|
|            MAR|  259|
|            TUR|  248|
|            HUN|  230|
+---------------+-----+
only showing top 30 rows



In [33]:
print ("Most and least frequent occurrences for adults:")
adultsOccurrenceDF = booking_df.groupBy("adults").count().sort(col("count").desc())
adultsOccurrenceDF.show()

Most and least frequent occurrences for adults:
+------+-----+
|adults|count|
+------+-----+
|     2|89680|
|     1|23027|
|     3| 6202|
|     0|  403|
|     4|   62|
|    26|    5|
|    27|    2|
|    20|    2|
|     5|    2|
|     6|    1|
|    55|    1|
|    40|    1|
|    50|    1|
|    10|    1|
+------+-----+



We noticed that there are **403 orders with 0 adults**, we'll keep this in mind and come back to investigate this later

In [34]:
print ("Most and least frequent occurrences for children:")
childrenOccurrenceDF = booking_df.groupBy("children").count().sort(col("count").desc())
childrenOccurrenceDF.show()

Most and least frequent occurrences for children:
+--------+------+
|children| count|
+--------+------+
|       0|110796|
|       1|  4861|
|       2|  3652|
|       3|    76|
|      NA|     4|
|      10|     1|
+--------+------+



We see here that we actually have null values here but in string `"NA"`, and the column is in string type instead of integer. 
We will replace them with 0 and cast the column to integers.

In [35]:
booking_df = booking_df.withColumn("children",when((col("children")=='NA'),None).otherwise(col("children")))

In [36]:
booking_df = booking_df.withColumn("children", booking_df["children"].cast(IntegerType()))

In [37]:
booking_df = booking_df.fillna(0, subset = ["children"])

In [38]:
print ("Most and least frequent occurrences for children:")
childrenOccurrenceDF = booking_df.groupBy("children").count().sort(col("count").desc())
childrenOccurrenceDF.show()

Most and least frequent occurrences for children:
+--------+------+
|children| count|
+--------+------+
|       0|110800|
|       1|  4861|
|       2|  3652|
|       3|    76|
|      10|     1|
+--------+------+



In [39]:
print ("Most and least frequent occurrences for babies:")
babiesOccurrenceDF = booking_df.groupBy("babies").count().sort(col("count").desc())
babiesOccurrenceDF.show()

Most and least frequent occurrences for babies:
+------+------+
|babies| count|
+------+------+
|     0|118473|
|     1|   900|
|     2|    15|
|     9|     1|
|    10|     1|
+------+------+



In [40]:
print ("Most and least frequent occurrences for is_repeated_guest:")
is_repeated_guestOccurrenceDF = booking_df.groupBy("is_repeated_guest").count().sort(col("count").desc())
is_repeated_guestOccurrenceDF.show()

Most and least frequent occurrences for is_repeated_guest:
+-----------------+------+
|is_repeated_guest| count|
+-----------------+------+
|                0|115580|
|                1|  3810|
+-----------------+------+



In [41]:
print ("Most and least frequent occurrences for previous_bookings_not_canceled:")
previous_bookings_not_canceledOccurrenceDF = booking_df.groupBy("previous_bookings_not_canceled").count().sort(col("count").desc())
previous_bookings_not_canceledOccurrenceDF.show()

Most and least frequent occurrences for previous_bookings_not_canceled:
+------------------------------+------+
|previous_bookings_not_canceled| count|
+------------------------------+------+
|                             0|115770|
|                             1|  1542|
|                             2|   580|
|                             3|   333|
|                             4|   229|
|                             5|   181|
|                             6|   115|
|                             7|    88|
|                             8|    70|
|                             9|    60|
|                            10|    53|
|                            11|    43|
|                            12|    37|
|                            13|    30|
|                            14|    28|
|                            15|    21|
|                            16|    20|
|                            25|    17|
|                            17|    16|
|                            18|    14|
+-------

In [42]:
print ("Most and least frequent occurrences for customer_type:")
customer_typeOccurrenceDF = booking_df.groupBy("customer_type").count().sort(col("count").desc())
customer_typeOccurrenceDF.show()

Most and least frequent occurrences for customer_type:
+---------------+-----+
|  customer_type|count|
+---------------+-----+
|      Transient|89613|
|Transient-Party|25124|
|       Contract| 4076|
|          Group|  577|
+---------------+-----+



According to the data source, the defintion of the customer_type is: 
- Contract - when the booking has an allotment or other type of contract associated to it; 
- Group – when the booking is associated to a group; 
- Transient – when the booking is not part of a group or contract, and is not associated to other transient booking; 
- Transient-party – when the booking is transient, but is associated to at least other transient booking

### C. Order related columns basic profiling

In [43]:
order_related_columns = ['hotel','market_segment' ,'distribution_channel', 'reserved_room_type','assigned_room_type',
                                'deposit_type', 'agent','lead_time','meal','days_in_waiting_list','adr','total_of_special_requests',
                                'required_car_parking_spaces','stays_in_weekend_nights','stays_in_week_nights', 'company']

In [44]:
print("Summary of order info related columns:")

booking_df.select(order_related_columns[:8]).summary().show()

Summary of order info related columns:
+-------+------------+--------------+--------------------+------------------+------------------+------------+------------------+------------------+
|summary|       hotel|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|             agent|         lead_time|
+-------+------------+--------------+--------------------+------------------+------------------+------------+------------------+------------------+
|  count|      119390|        119390|              119390|            119390|            119390|      119390|            103050|            119390|
|   mean|        null|          null|                null|              null|              null|        null| 86.69338185346919|104.01141636652986|
| stddev|        null|          null|                null|              null|              null|        null|110.77454764295116| 106.8630970479881|
|    min|  City Hotel|      Aviation|           Corporate|               

In [45]:
print("Summary of order info related columns:")

booking_df.select(order_related_columns[8:12]).summary().show()

Summary of order info related columns:
+-------+---------+--------------------+------------------+-------------------------+
|summary|     meal|days_in_waiting_list|               adr|total_of_special_requests|
+-------+---------+--------------------+------------------+-------------------------+
|  count|   119390|              119390|            119390|                   119390|
|   mean|     null|   2.321149174972778|101.83112153446218|       0.5713627607002262|
| stddev|     null|  17.594720878776243| 50.53579028554872|       0.7927984228094107|
|    min|       BB|                   0|             -6.38|                        0|
|    25%|     null|                   0|             69.29|                        0|
|    50%|     null|                   0|              94.5|                        0|
|    75%|     null|                   0|             126.0|                        1|
|    max|Undefined|                 391|            5400.0|                        5|
+-------+------

In [46]:
print("Summary of order related columns:")

booking_df.select(order_related_columns[12:]).summary().show()

Summary of order related columns:
+-------+---------------------------+-----------------------+--------------------+------------------+
|summary|required_car_parking_spaces|stays_in_weekend_nights|stays_in_week_nights|           company|
+-------+---------------------------+-----------------------+--------------------+------------------+
|  count|                     119390|                 119390|              119390|              6797|
|   mean|        0.06251779881062065|     0.9275986263506156|   2.500301532791691|189.26673532440782|
| stddev|        0.24529114746749414|     0.9986134945978791|  1.9082856150479042| 131.6550146385122|
|    min|                          0|                      0|                   0|                10|
|    25%|                          0|                      0|                   1|              62.0|
|    50%|                          0|                      1|                   2|             179.0|
|    75%|                          0|           

In [47]:
print("Checking for nulls in the time related columns:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in order_related_columns[0:8]]).show()

Checking for nulls in the time related columns:
+-----+--------------+--------------------+------------------+------------------+------------+-----+---------+
|hotel|market_segment|distribution_channel|reserved_room_type|assigned_room_type|deposit_type|agent|lead_time|
+-----+--------------+--------------------+------------------+------------------+------------+-----+---------+
|    0|             0|                   0|                 0|                 0|           0|16340|        0|
+-----+--------------+--------------------+------------------+------------------+------------+-----+---------+



In [48]:
print("Checking for nulls in the time related columns:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in order_related_columns[8:]]).show()

Checking for nulls in the time related columns:
+----+--------------------+---+-------------------------+---------------------------+-----------------------+--------------------+-------+
|meal|days_in_waiting_list|adr|total_of_special_requests|required_car_parking_spaces|stays_in_weekend_nights|stays_in_week_nights|company|
+----+--------------------+---+-------------------------+---------------------------+-----------------------+--------------------+-------+
|   0|                   0|  0|                        0|                          0|                      0|                   0| 112593|
+----+--------------------+---+-------------------------+---------------------------+-----------------------+--------------------+-------+



We could see that in the columns `agent` & `company` there are a lot of null values, let's inspect these columns further:

In [49]:
print("Checking for unique values in agent & company column:")

booking_df.select([countDistinct(c).alias(c) for c in ["agent", "company"]]).show()

Checking for unique values in agent & company column:
+-----+-------+
|agent|company|
+-----+-------+
|  333|    352|
+-----+-------+



We could see that for `agent` and `company` columns they don't have 0 in their values from the summaries, so we could recode the NULLs into 0s. 

In [50]:
booking_df = booking_df.fillna("0", subset=['agent', 'company'])

In [51]:
booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in ["agent","company"]]).show()

+-----+-------+
|agent|company|
+-----+-------+
|    0|      0|
+-----+-------+



In [52]:
print ("Most and least frequent occurrences for agent:")
agentOccurrenceDF = booking_df.groupBy("agent").count().sort(col("count").desc())
agentOccurrenceDF.show()

Most and least frequent occurrences for agent:
+-----+-----+
|agent|count|
+-----+-----+
|    9|31961|
|    0|16340|
|  240|13922|
|    1| 7191|
|   14| 3640|
|    7| 3539|
|    6| 3290|
|  250| 2870|
|  241| 1721|
|   28| 1666|
|    8| 1514|
|    3| 1336|
|   37| 1230|
|   19| 1061|
|   40| 1039|
|  314|  927|
|   21|  875|
|  229|  786|
|  242|  780|
|   83|  696|
+-----+-----+
only showing top 20 rows



In [53]:
print ("Most and least frequent occurrences for company:")
companyOccurrenceDF = booking_df.groupBy("company").count().sort(col("count").desc())
companyOccurrenceDF.show()

Most and least frequent occurrences for company:
+-------+------+
|company| count|
+-------+------+
|      0|112593|
|     40|   927|
|    223|   784|
|     67|   267|
|     45|   250|
|    153|   215|
|    174|   149|
|    219|   141|
|    281|   138|
|    154|   133|
|    405|   119|
|    233|   114|
|     51|    99|
|     94|    87|
|     47|    72|
|    135|    66|
|    169|    65|
|    242|    62|
|    331|    61|
|    348|    59|
+-------+------+
only showing top 20 rows



In [54]:
print ("Most and least frequent occurrences for hotel:")
hotelOccurrenceDF = booking_df.groupBy("hotel").count().sort(col("count").desc())
hotelOccurrenceDF.show()

Most and least frequent occurrences for hotel:
+------------+-----+
|       hotel|count|
+------------+-----+
|  City Hotel|79330|
|Resort Hotel|40060|
+------------+-----+



In [55]:
print ("Most and least frequent occurrences for market_segment:")
market_segmentOccurrenceDF = booking_df.groupBy("market_segment").count().sort(col("count").desc())
market_segmentOccurrenceDF.show()

Most and least frequent occurrences for market_segment:
+--------------+-----+
|market_segment|count|
+--------------+-----+
|     Online TA|56477|
| Offline TA/TO|24219|
|        Groups|19811|
|        Direct|12606|
|     Corporate| 5295|
| Complementary|  743|
|      Aviation|  237|
|     Undefined|    2|
+--------------+-----+



In [56]:
print ("Most and least frequent occurrences for distribution_channel:")
distribution_channelOccurrenceDF = booking_df.groupBy("distribution_channel").count().sort(col("count").desc())
distribution_channelOccurrenceDF.show()

Most and least frequent occurrences for distribution_channel:
+--------------------+-----+
|distribution_channel|count|
+--------------------+-----+
|               TA/TO|97870|
|              Direct|14645|
|           Corporate| 6677|
|                 GDS|  193|
|           Undefined|    5|
+--------------------+-----+



In [57]:
print ("Most and least frequent occurrences for reserved_room_type:")
reserved_room_typeOccurrenceDF = booking_df.groupBy("reserved_room_type").count().sort(col("count").desc())
reserved_room_typeOccurrenceDF.show()

Most and least frequent occurrences for reserved_room_type:
+------------------+-----+
|reserved_room_type|count|
+------------------+-----+
|                 A|85994|
|                 D|19201|
|                 E| 6535|
|                 F| 2897|
|                 G| 2094|
|                 B| 1118|
|                 C|  932|
|                 H|  601|
|                 P|   12|
|                 L|    6|
+------------------+-----+



In [58]:
print ("Most and least frequent occurrences for assigned_room_type:")
assigned_room_typeOccurrenceDF = booking_df.groupBy("assigned_room_type").count().sort(col("count").desc())
assigned_room_typeOccurrenceDF.show()

Most and least frequent occurrences for assigned_room_type:
+------------------+-----+
|assigned_room_type|count|
+------------------+-----+
|                 A|74053|
|                 D|25322|
|                 E| 7806|
|                 F| 3751|
|                 G| 2553|
|                 C| 2375|
|                 B| 2163|
|                 H|  712|
|                 I|  363|
|                 K|  279|
|                 P|   12|
|                 L|    1|
+------------------+-----+



In [59]:
print ("Most and least frequent occurrences for deposit_type:")
deposit_typeOccurrenceDF = booking_df.groupBy("deposit_type").count().sort(col("count").desc())
deposit_typeOccurrenceDF.show()

Most and least frequent occurrences for deposit_type:
+------------+------+
|deposit_type| count|
+------------+------+
|  No Deposit|104641|
|  Non Refund| 14587|
|  Refundable|   162|
+------------+------+



In [60]:
print ("Most and least frequent occurrences for lead_time:")
lead_timeOccurrenceDF = booking_df.groupBy("lead_time").count().sort(col("count").desc())
lead_timeOccurrenceDF.show()

Most and least frequent occurrences for lead_time:
+---------+-----+
|lead_time|count|
+---------+-----+
|        0| 6345|
|        1| 3460|
|        2| 2069|
|        3| 1816|
|        4| 1715|
|        5| 1565|
|        6| 1445|
|        7| 1331|
|        8| 1138|
|       12| 1079|
|       11| 1055|
|        9|  992|
|       10|  976|
|       14|  965|
|       16|  942|
|       17|  881|
|       19|  839|
|       15|  839|
|       34|  828|
|       18|  826|
+---------+-----+
only showing top 20 rows



In [61]:
print ("Most and least frequent occurrences for meal:")
mealOccurrenceDF = booking_df.groupBy("meal").count().sort(col("count").desc())
mealOccurrenceDF.show()

Most and least frequent occurrences for meal:
+---------+-----+
|     meal|count|
+---------+-----+
|       BB|92310|
|       HB|14463|
|       SC|10650|
|Undefined| 1169|
|       FB|  798|
+---------+-----+



According to the data source, the defintion of the types of meal are:

- Undefined/SC – no meal package; 
- BB – Bed & Breakfast; 
- HB – Half board (breakfast and one other meal – usually dinner); 
- FB – Full board (breakfast, lunch and dinner)

In [62]:
print ("Most and least frequent occurrences for adr:")
adrOccurrenceDF = booking_df.groupBy("adr").count().sort(col("count").desc())
adrOccurrenceDF.show()

Most and least frequent occurrences for adr:
+-----+-----+
|  adr|count|
+-----+-----+
| 62.0| 3754|
| 75.0| 2715|
| 90.0| 2473|
| 65.0| 2418|
|  0.0| 1959|
| 80.0| 1889|
| 95.0| 1661|
|120.0| 1607|
|100.0| 1573|
| 85.0| 1538|
|110.0| 1525|
| 60.0| 1313|
|130.0| 1275|
| 48.0| 1123|
|115.0| 1080|
| 70.0| 1044|
| 99.0|  905|
|140.0|  866|
|126.0|  852|
|108.0|  818|
+-----+-----+
only showing top 20 rows



`adr` stands for **Average Daily Rate**, which is calculated by the sum of revenue from the booking divided by the length of stay.

Because our data consists of booking from 2 hotels, we could see they are quite some repetive values in `adr`.

We could further bin the price into categories, and investigate those bookings with an average daily rate of 0.

In [63]:
print ("Most and least frequent occurrences for total_of_special_requests:")
total_of_special_requestsOccurrenceDF = booking_df.groupBy("total_of_special_requests").count().sort(col("count").desc())
total_of_special_requestsOccurrenceDF.show()

Most and least frequent occurrences for total_of_special_requests:
+-------------------------+-----+
|total_of_special_requests|count|
+-------------------------+-----+
|                        0|70318|
|                        1|33226|
|                        2|12969|
|                        3| 2497|
|                        4|  340|
|                        5|   40|
+-------------------------+-----+



In [64]:
print ("Most and least frequent occurrences for required_car_parking_spaces:")
required_car_parking_spacesOccurrenceDF = booking_df.groupBy("required_car_parking_spaces").count().sort(col("count").desc())
required_car_parking_spacesOccurrenceDF.show()

Most and least frequent occurrences for required_car_parking_spaces:
+---------------------------+------+
|required_car_parking_spaces| count|
+---------------------------+------+
|                          0|111974|
|                          1|  7383|
|                          2|    28|
|                          3|     3|
|                          8|     2|
+---------------------------+------+



In [65]:
print ("Most and least frequent occurrences for stays_in_weekend_nights:")
stays_in_weekend_nightsOccurrenceDF = booking_df.groupBy("stays_in_weekend_nights").count().sort(col("count").desc())
stays_in_weekend_nightsOccurrenceDF.show()

Most and least frequent occurrences for stays_in_weekend_nights:
+-----------------------+-----+
|stays_in_weekend_nights|count|
+-----------------------+-----+
|                      0|51998|
|                      2|33308|
|                      1|30626|
|                      4| 1855|
|                      3| 1259|
|                      6|  153|
|                      5|   79|
|                      8|   60|
|                      7|   19|
|                      9|   11|
|                     10|    7|
|                     12|    5|
|                     13|    3|
|                     16|    3|
|                     14|    2|
|                     19|    1|
|                     18|    1|
+-----------------------+-----+



In [66]:
print ("Most and least frequent occurrences for stays_in_week_nights:")
stays_in_week_nightsOccurrenceDF = booking_df.groupBy("stays_in_week_nights").count().sort(col("count").desc())
stays_in_week_nightsOccurrenceDF.show()

Most and least frequent occurrences for stays_in_week_nights:
+--------------------+-----+
|stays_in_week_nights|count|
+--------------------+-----+
|                   2|33684|
|                   1|30310|
|                   3|22258|
|                   5|11077|
|                   4| 9563|
|                   0| 7645|
|                   6| 1499|
|                  10| 1036|
|                   7| 1029|
|                   8|  656|
|                   9|  231|
|                  15|   85|
|                  11|   56|
|                  19|   44|
|                  12|   42|
|                  20|   41|
|                  14|   35|
|                  13|   27|
|                  16|   16|
|                  21|   15|
+--------------------+-----+
only showing top 20 rows



### D. Issue related columns basic profiling

In [67]:
issue_related_columns = ["is_canceled", "previous_cancellations", "booking_changes", "reservation_status"]

In [68]:
print("Summary of issue related columns:")

booking_df.select(issue_related_columns).summary().show()

Summary of issue related columns:
+-------+-------------------+----------------------+-------------------+------------------+
|summary|        is_canceled|previous_cancellations|    booking_changes|reservation_status|
+-------+-------------------+----------------------+-------------------+------------------+
|  count|             119390|                119390|             119390|            119390|
|   mean|0.37041628277075134|   0.08711784906608594|0.22112404724013737|              null|
| stddev|0.48291822659259803|    0.8443363841545121| 0.6523055726747705|              null|
|    min|                  0|                     0|                  0|          Canceled|
|    25%|                  0|                     0|                  0|              null|
|    50%|                  0|                     0|                  0|              null|
|    75%|                  1|                     0|                  0|              null|
|    max|                  1|                 

In [69]:
print("Checking for nulls in the issue related columns:")

booking_df.select([count(when(col(c).isNull(), c)).alias(c) for c in issue_related_columns]).show()

Checking for nulls in the issue related columns:
+-----------+----------------------+---------------+------------------+
|is_canceled|previous_cancellations|booking_changes|reservation_status|
+-----------+----------------------+---------------+------------------+
|          0|                     0|              0|                 0|
+-----------+----------------------+---------------+------------------+



In [70]:
print ("Most and least frequent occurrences for is_canceled:")
is_canceledOccurrenceDF = booking_df.groupBy("is_canceled").count().sort(col("count").desc())
is_canceledOccurrenceDF.show()

Most and least frequent occurrences for is_canceled:
+-----------+-----+
|is_canceled|count|
+-----------+-----+
|          0|75166|
|          1|44224|
+-----------+-----+



In [71]:
print ("Most and least frequent occurrences for previous_cancellations:")
previous_cancellationsOccurrenceDF = booking_df.groupBy("previous_cancellations").count().sort(col("count").desc())
previous_cancellationsOccurrenceDF.show()

Most and least frequent occurrences for previous_cancellations:
+----------------------+------+
|previous_cancellations| count|
+----------------------+------+
|                     0|112906|
|                     1|  6051|
|                     2|   116|
|                     3|    65|
|                    24|    48|
|                    11|    35|
|                     4|    31|
|                    26|    26|
|                    25|    25|
|                     6|    22|
|                     5|    19|
|                    19|    19|
|                    14|    14|
|                    13|    12|
|                    21|     1|
+----------------------+------+



In [72]:
print ("Most and least frequent occurrences for booking_changes:")
booking_changesOccurrenceDF = booking_df.groupBy("booking_changes").count().sort(col("count").desc())
booking_changesOccurrenceDF.show()

Most and least frequent occurrences for booking_changes:
+---------------+------+
|booking_changes| count|
+---------------+------+
|              0|101314|
|              1| 12701|
|              2|  3805|
|              3|   927|
|              4|   376|
|              5|   118|
|              6|    63|
|              7|    31|
|              8|    17|
|              9|     8|
|             10|     6|
|             13|     5|
|             14|     5|
|             15|     3|
|             12|     2|
|             16|     2|
|             11|     2|
|             17|     2|
|             20|     1|
|             21|     1|
+---------------+------+
only showing top 20 rows



In [73]:
print ("Most and least frequent occurrences for reservation_status:")
reservation_statusOccurrenceDF = booking_df.groupBy("reservation_status").count().sort(col("count").desc())
reservation_statusOccurrenceDF.show()

Most and least frequent occurrences for reservation_status:
+------------------+-----+
|reservation_status|count|
+------------------+-----+
|         Check-Out|75166|
|          Canceled|43017|
|           No-Show| 1207|
+------------------+-----+



In [74]:
43017 + 1207

44224

By adding `Canceled` and `No-Show`, we get the count of `is_canceld`. In other words, if we want further details on the cancellation of an order, we could use `reservation_status`.

## 5. Answer some business questions to understand the possible factors of cancelation

### A. Create and new columns & segmentations of the dataset

We will create the following new columns:
- **total_headcount**: `adults` + `children` + `babies`
- **guest_size**: bin the `total_headcount` into 4 categories: *solo*, *couple*, *small_group*, *big_group*
- **is_family**: if `children` or `babies` is not 0
- **is_agent**: if `agent` != 0 (we've recoded the null values in this column to 0)
- **is_company**: if `company` != 0 (we've recoded the null values in this column to 0)
- **adr_cat**: bin the `adr` column into 3 categories: *below_average*, *average*, *above_average*
- **lead_time_cat**: bin the `lead_time` into 4 categories: *same_day*, *short*, *medium*, *long*
- **total_stays**: `stays_in_weekend_nights` + `stays_in_week_nights`
- **stay_length_cat**: bin the `total_stays` into 5 categories: *one*, *two*, *three*, *medium*, *long*
- **booking_change_cat**: bin `booking_changes` into 4 categories: *none*, *one*, *some*, *many*


We will break the current DF into a two smaller ones to better understand each hotel.
- **cityHotelDF**: orders for city hotel
- **resortHotelDF**: orders for resort hotel


Because `adr_cat` will be more hotel specific, we will create the column after spliting the dataset.

In [75]:
booking_df = booking_df.withColumn("total_headcount", booking_df.adults + booking_df.children + booking_df.babies)

In [76]:
booking_df.select("total_headcount").summary().show()

+-------+------------------+
|summary|   total_headcount|
+-------+------------------+
|  count|            119390|
|   mean|1.9682385459418712|
| stddev|0.7223942426586695|
|    min|                 0|
|    25%|                 2|
|    50%|                 2|
|    75%|                 2|
|    max|                55|
+-------+------------------+



In [77]:
booking_df.groupBy("total_headcount").count().sort(col("count").desc()).show()

+---------------+-----+
|total_headcount|count|
+---------------+-----+
|              2|82051|
|              1|22581|
|              3|10495|
|              4| 3929|
|              0|  180|
|              5|  137|
|             26|    5|
|             27|    2|
|             12|    2|
|             20|    2|
|             10|    2|
|              6|    1|
|             55|    1|
|             40|    1|
|             50|    1|
+---------------+-----+



In [78]:
booking_df = booking_df.withColumn("guest_size", when(col("total_headcount") == 1, "1. solo")
                                                .when(col("total_headcount") == 2, "2. couple")
                                                .when((col("total_headcount") >2) & (col("total_headcount") < 6), "3. small_group")
                                                .otherwise("4. big_group")
                               )

In [79]:
booking_df.groupBy("guest_size").count().sort(col("count").desc()).show()

+--------------+-----+
|    guest_size|count|
+--------------+-----+
|     2. couple|82051|
|       1. solo|22581|
|3. small_group|14561|
|  4. big_group|  197|
+--------------+-----+



In [80]:
booking_df.groupBy("hotel","guest_size").count().sort(col("guest_size")).show()

+------------+--------------+-----+
|       hotel|    guest_size|count|
+------------+--------------+-----+
|Resort Hotel|       1. solo| 7013|
|  City Hotel|       1. solo|15568|
|  City Hotel|     2. couple|53935|
|Resort Hotel|     2. couple|28116|
|Resort Hotel|3. small_group| 4903|
|  City Hotel|3. small_group| 9658|
|  City Hotel|  4. big_group|  169|
|Resort Hotel|  4. big_group|   28|
+------------+--------------+-----+



In [81]:
booking_df = booking_df.withColumn("is_family", when((booking_df.children + booking_df.babies) > 0, 1)
                                .otherwise(0))

In [82]:
booking_df = booking_df.withColumn("is_agent", when(booking_df.agent > 0, "1")
                                .otherwise(0))

In [83]:
booking_df = booking_df.withColumn("is_company", when(booking_df.company > 0, 1)
                                .otherwise(0))

In [84]:
booking_df = booking_df.withColumn("total_stays", booking_df.stays_in_weekend_nights + booking_df.stays_in_week_nights)

In [85]:
booking_df.groupBy("total_stays").count().sort(col("count").desc()).show()

+-----------+-----+
|total_stays|count|
+-----------+-----+
|          2|27643|
|          3|27076|
|          1|21020|
|          4|17383|
|          7| 8655|
|          5| 7784|
|          6| 3857|
|          8| 1161|
|         10| 1139|
|         14|  916|
|          9|  841|
|          0|  715|
|         11|  396|
|         12|  223|
|         13|  142|
|         15|   75|
|         21|   71|
|         16|   40|
|         25|   37|
|         28|   35|
+-----------+-----+
only showing top 20 rows



In [86]:
booking_df.select("total_stays").summary().show()

+-------+------------------+
|summary|       total_stays|
+-------+------------------+
|  count|            119390|
|   mean|3.4279001591423066|
| stddev| 2.557438669051983|
|    min|                 0|
|    25%|                 2|
|    50%|                 3|
|    75%|                 4|
|    max|                69|
+-------+------------------+



In [87]:
booking_df = booking_df.withColumn("stay_length_cat", when(col("total_stays") == 1, "1. one")
                                                     .when(col("total_stays") == 2, "2. two")
                                                     .when(col("total_stays") == 3, "3. three")
                                                     .when((col("total_stays") > 3 ) & (col("total_stays") < 7), "4. medium")
                                                     .otherwise("5. long")
                               )

In [88]:
booking_df.groupBy("stay_length_cat").count().sort(col("count").desc()).show()

+---------------+-----+
|stay_length_cat|count|
+---------------+-----+
|      4. medium|29024|
|         2. two|27643|
|       3. three|27076|
|         1. one|21020|
|        5. long|14627|
+---------------+-----+



In [89]:
booking_df.select("lead_time").summary().show()

+-------+------------------+
|summary|         lead_time|
+-------+------------------+
|  count|            119390|
|   mean|104.01141636652986|
| stddev| 106.8630970479881|
|    min|                 0|
|    25%|                18|
|    50%|                69|
|    75%|               160|
|    max|               737|
+-------+------------------+



In [90]:
booking_df.groupBy("lead_time").count().sort(col("count").desc()).show()

+---------+-----+
|lead_time|count|
+---------+-----+
|        0| 6345|
|        1| 3460|
|        2| 2069|
|        3| 1816|
|        4| 1715|
|        5| 1565|
|        6| 1445|
|        7| 1331|
|        8| 1138|
|       12| 1079|
|       11| 1055|
|        9|  992|
|       10|  976|
|       14|  965|
|       16|  942|
|       17|  881|
|       15|  839|
|       19|  839|
|       34|  828|
|       18|  826|
+---------+-----+
only showing top 20 rows



In [91]:
booking_df = booking_df.withColumn("lead_time_cat", when(col("lead_time") == 0, "1. same_day")
                                      .when((col("lead_time") > 0 ) & (col("lead_time") <= 7), "2. short")
                                      .when((col("lead_time") > 7 ) & (col("lead_time") <= 90), "3. medium")
                                      .when((col("lead_time") > 90 ) & (col("lead_time") <= 180), "4. long")
                                      .otherwise("5. super_long")
                               )

In [92]:
booking_df.groupBy("lead_time_cat").count().sort(col("count").desc()).show()

+-------------+-----+
|lead_time_cat|count|
+-------------+-----+
|    3. medium|48513|
|      4. long|26439|
|5. super_long|24692|
|     2. short|13401|
|  1. same_day| 6345|
+-------------+-----+



In [93]:
booking_df.select("booking_changes").summary().show()

+-------+-------------------+
|summary|    booking_changes|
+-------+-------------------+
|  count|             119390|
|   mean|0.22112404724013737|
| stddev| 0.6523055726747705|
|    min|                  0|
|    25%|                  0|
|    50%|                  0|
|    75%|                  0|
|    max|                 21|
+-------+-------------------+



In [94]:
booking_df.groupBy("booking_changes").count().sort(col("count").desc()).show()

+---------------+------+
|booking_changes| count|
+---------------+------+
|              0|101314|
|              1| 12701|
|              2|  3805|
|              3|   927|
|              4|   376|
|              5|   118|
|              6|    63|
|              7|    31|
|              8|    17|
|              9|     8|
|             10|     6|
|             13|     5|
|             14|     5|
|             15|     3|
|             12|     2|
|             16|     2|
|             11|     2|
|             17|     2|
|             20|     1|
|             21|     1|
+---------------+------+
only showing top 20 rows



In [95]:
booking_df = booking_df.withColumn("booking_change_cat", when(col("booking_changes") == 0, "1. none")
                                      .when(col("lead_time")  == 1, "2. one")
                                      .when((col("lead_time") > 1 ) & (col("lead_time") <= 5), "3. some")
                                      .otherwise("4. many")
                                   )

In [96]:
booking_df = booking_df.withColumn("season_cat", when((col("arrival_date_month") == 8) | 
                                         (col("arrival_date_month") == 7) |
                                         (col("arrival_date_month") == 5) |
                                         (col("arrival_date_month") == 10),"1. high season")
                                   .when((col("arrival_date_month") == 2) | 
                                         (col("arrival_date_month") == 11) |
                                         (col("arrival_date_month") == 12) |
                                         (col("arrival_date_month") == 1),"3. low season")
                                   .otherwise("2. other"))
                               

In [97]:
booking_df.groupBy("season_cat").count().sort(col("count").desc()).show()

+--------------+-----+
|    season_cat|count|
+--------------+-----+
|1. high season|49489|
|      2. other|42330|
| 3. low season|27571|
+--------------+-----+



---

In [98]:
cityHotelDF = booking_df.where(col("hotel") == "City Hotel").cache()

In [99]:
resortHotelDF = booking_df.where(col("hotel") == "Resort Hotel").cache()

In [100]:
cityHotelDF.select("adr").summary().show()

+-------+------------------+
|summary|               adr|
+-------+------------------+
|  count|             79330|
|   mean|105.30446539770196|
| stddev| 43.60295383709462|
|    min|               0.0|
|    25%|              79.2|
|    50%|              99.9|
|    75%|             126.0|
|    max|            5400.0|
+-------+------------------+



In [101]:
cityHotelDF = cityHotelDF.withColumn("adr_cat", when(col("adr") < 80, "1. below_average")
                                               .when((col("adr") >= 80) & (col("adr") <= 125), "2. average")
                                               .otherwise("3. above_average"))             

In [102]:
cityHotelDF.groupBy("adr_cat").count().orderBy("adr_cat").show()

+----------------+-----+
|         adr_cat|count|
+----------------+-----+
|1. below_average|20309|
|      2. average|38559|
|3. above_average|20462|
+----------------+-----+



In [103]:
resortHotelDF.select("adr").summary().show()

+-------+-----------------+
|summary|              adr|
+-------+-----------------+
|  count|            40060|
|   mean|94.95292960559225|
| stddev|61.44241797380695|
|    min|            -6.38|
|    25%|             50.0|
|    50%|             75.0|
|    75%|            125.0|
|    max|            508.0|
+-------+-----------------+



In [104]:
resortHotelDF = resortHotelDF.withColumn("adr_cat", when(col("adr") < 50, "1. below_average")
                                                   .when((col("adr") >= 50) & (col("adr") <= 125), "2. average")
                                                   .otherwise("3. above_average"))    

In [105]:
resortHotelDF.groupBy("adr_cat").count().orderBy("adr_cat").show()

+----------------+-----+
|         adr_cat|count|
+----------------+-----+
|1. below_average| 9724|
|      2. average|20350|
|3. above_average| 9986|
+----------------+-----+



---

### B. Comparsion across dimensions

Dimension to compare:
- Canceled vs not-canceled orders
- City vs resort orders


We will first compare all the numneric columns between the dimensions, 
The numeric columns are:
- `lead_time`,`stays_in_weekend_nights`,`stays_in_week_nights`,`total_stays`, 
- `adults`,`children`,`adr`,`babies`,`total_headcount`,
- `previous_cancellations`,`previous_bookings_not_canceled`,`booking_changes`, `days_in_waiting_list`, 
- `total_of_special_requests`, `required_car_parking_spaces`


In [119]:
booking_df.groupby("hotel","is_canceled") \
           .mean("lead_time","stays_in_weekend_nights","stays_in_week_nights","total_stays")\
           .sort("hotel") \
           .show()

+------------+-----------+------------------+----------------------------+-------------------------+------------------+
|       hotel|is_canceled|    avg(lead_time)|avg(stays_in_weekend_nights)|avg(stays_in_week_nights)|  avg(total_stays)|
+------------+-----------+------------------+----------------------------+-------------------------+------------------+
|  City Hotel|          0| 80.70273427360041|          0.8006835684001038|       2.1229341524617116|2.9236177208618153|
|  City Hotel|          1|150.28122167844845|           0.787505286689626|       2.2667814633556884|3.0542867500453146|
|Resort Hotel|          1|128.68054306779356|          1.3352814242042799|       3.4402985074626864| 4.775579931666966|
|Resort Hotel|          0| 78.83761835648627|           1.133906973529615|        3.008984725965858| 4.142891699495473|
+------------+-----------+------------------+----------------------------+-------------------------+------------------+



In [120]:
booking_df.groupby("hotel","is_canceled") \
           .mean("adults", "children","babies", "total_headcount")\
           .sort("hotel") \
           .show()

+------------+-----------+------------------+-------------------+--------------------+--------------------+
|       hotel|is_canceled|       avg(adults)|      avg(children)|         avg(babies)|avg(total_headcount)|
+------------+-----------+------------------+-------------------+--------------------+--------------------+
|  City Hotel|          1|1.8829073771977525|0.07945139266509577|0.001933417920367...|  1.9642921877832156|
|  City Hotel|          0|1.8281128320498399|0.09989616682530068|0.007095266937786623|  1.9351042658129272|
|Resort Hotel|          0|1.8323311908217568|0.10626166286543645|0.015619600525260903|  1.9542124542124542|
|Resort Hotel|          1|1.9577414134148534|0.18701672361086136|0.009440748066894444|   2.154198885092609|
+------------+-----------+------------------+-------------------+--------------------+--------------------+



In [121]:
booking_df.groupby("hotel","is_canceled") \
           .mean("previous_cancellations", "previous_bookings_not_canceled", "booking_changes", "days_in_waiting_list")\
           .sort("hotel") \
           .show()

+------------+-----------+---------------------------+-----------------------------------+--------------------+-------------------------+
|       hotel|is_canceled|avg(previous_cancellations)|avg(previous_bookings_not_canceled)|avg(booking_changes)|avg(days_in_waiting_list)|
+------------+-----------+---------------------------+-----------------------------------+--------------------+-------------------------+
|  City Hotel|          0|       0.021156009344985724|                 0.2085099939430648|  0.2643635891667388|       2.1500822012633036|
|  City Hotel|          1|        0.16156123497069663|               0.026040722614947737| 0.07984411818017038|        4.730409038728777|
|Resort Hotel|          0|       0.007222337411016656|                0.19413919413919414|  0.3396917547860944|       0.6949339968207893|
|Resort Hotel|          1|        0.34759935263441827|               0.022388059701492536| 0.15338967811544688|      0.09278906671461967|
+------------+-----------+--------

In [122]:
booking_df.groupby("hotel","is_canceled") \
           .mean("total_of_special_requests", "required_car_parking_spaces")\
           .sort("hotel") \
           .show()

+------------+-----------+------------------------------+--------------------------------+
|       hotel|is_canceled|avg(total_of_special_requests)|avg(required_car_parking_spaces)|
+------------+-----------+------------------------------+--------------------------------+
|  City Hotel|          0|            0.7410876525049753|            0.041814484727870554|
|  City Hotel|          1|            0.2757537308923932|                             0.0|
|Resort Hotel|          0|            0.6708825765429539|              0.1911327666044647|
|Resort Hotel|          1|            0.4867829527063478|                             0.0|
+------------+-----------+------------------------------+--------------------------------+



We could see that in the resort hotel it tends to have more children & babies, whilst having more booking_changes

---

We will then investigate into other dimensions, comparing their cancelation rate:
    
- Guests with kids or not, also the number of guest
- Booking through Company & customer type
- Distribution channel, market segement & agents
- Nationality
- Meals
- Lead time & number of changes made the booking
- Lead time & Price
- Lead time & deposit

In [123]:
def cancelation_ratio(df, list_of_columns):
    df1 = df.groupby(list_of_columns) \
          .sum("is_canceled")\
          .orderBy(list_of_columns) \
          .select(col("sum(is_canceled)").alias("cancel_count"))
    
    df2 = df.groupby(list_of_columns) \
          .count()\
          .orderBy(list_of_columns)
    
    df1 = df1.withColumn("row_id", monotonically_increasing_id())

    df2 = df2.withColumn("row_id", monotonically_increasing_id())

    result_df = df2.join(df1, ("row_id")).drop("row_id")
    result_df = result_df.withColumn("cancelation_ratio", round(col("cancel_count")/col("count"), 4))
    
    return(result_df)   

---

We first obtain the baseline of cancelation for each hotel.

In [124]:
column_list = ['hotel']
hotel_df = cancelation_ratio(booking_df, column_list)
hotel_df.show()

+------------+-----+------------+-----------------+
|       hotel|count|cancel_count|cancelation_ratio|
+------------+-----+------------+-----------------+
|  City Hotel|79330|       33102|           0.4173|
|Resort Hotel|40060|       11122|           0.2776|
+------------+-----+------------+-----------------+



---

In [125]:
hotel_company_customer_type = ['hotel', 'is_company', 'customer_type']
hotel_company_customer_df = cancelation_ratio(booking_df, hotel_company_customer_type)
hotel_company_customer_df.show()

+------------+----------+---------------+-----+------------+-----------------+
|       hotel|is_company|  customer_type|count|cancel_count|cancelation_ratio|
+------------+----------+---------------+-----+------------+-----------------+
|  City Hotel|         0|       Contract| 2298|        1105|           0.4809|
|  City Hotel|         0|          Group|  276|          26|           0.0942|
|  City Hotel|         0|      Transient|56840|       26526|           0.4667|
|  City Hotel|         0|Transient-Party|16227|        4661|           0.2872|
|  City Hotel|         1|       Contract|    2|           0|              0.0|
|  City Hotel|         1|          Group|   17|           3|           0.1765|
|  City Hotel|         1|      Transient| 2564|         572|           0.2231|
|  City Hotel|         1|Transient-Party| 1106|         209|            0.189|
|Resort Hotel|         0|       Contract| 1773|         157|           0.0886|
|Resort Hotel|         0|          Group|  252|     

Group & Transient group orders have much lower cancelation rate across both hotels, while contract orders, though small, have a high cancellation rate especially for leisure travellers (not paid by companies) in the city hotel. 


---

In [126]:
hotel_is_family_guest_size = ['hotel','is_family','guest_size']
hotel_is_family_guest_size_df = cancelation_ratio(booking_df, hotel_is_family_guest_size)
hotel_is_family_guest_size_df.show()

+------------+---------+--------------+-----+------------+-----------------+
|       hotel|is_family|    guest_size|count|cancel_count|cancelation_ratio|
+------------+---------+--------------+-----+------------+-----------------+
|  City Hotel|        0|       1. solo|15564|        5372|           0.3452|
|  City Hotel|        0|     2. couple|53526|       24090|           0.4501|
|  City Hotel|        0|3. small_group| 4670|        1755|           0.3758|
|  City Hotel|        0|  4. big_group|  167|          23|           0.1377|
|  City Hotel|        1|       1. solo|    4|           0|              0.0|
|  City Hotel|        1|     2. couple|  409|         129|           0.3154|
|  City Hotel|        1|3. small_group| 4988|        1733|           0.3474|
|  City Hotel|        1|  4. big_group|    2|           0|              0.0|
|Resort Hotel|        0|       1. solo| 7013|        1183|           0.1687|
|Resort Hotel|        0|     2. couple|28034|        8334|           0.2973|

Non-family couple travellers (cannot assume relationship, just mean it’s a party size of 2) are the most likely to cancel for the city hotel, maybe due to the fact that they are the biggest composition in terms of demographic for this hotel.

For the resort hotel, the most likely to cancel are the non-family big groups, though they are little in number of orders. The more significant demographic which is prone to cancel is small families with children.

---

In [127]:
column_list = ['hotel','is_agent']
hotel_agent_df = cancelation_ratio(booking_df, column_list)
hotel_agent_df.show()

+------------+--------+-----+------------+-----------------+
|       hotel|is_agent|count|cancel_count|cancelation_ratio|
+------------+--------+-----+------------+-----------------+
|  City Hotel|       0| 8131|        2609|           0.3209|
|  City Hotel|       1|71199|       30493|           0.4283|
|Resort Hotel|       0| 8209|        1421|           0.1731|
|Resort Hotel|       1|31851|        9701|           0.3046|
+------------+--------+-----+------------+-----------------+



We could observe that orders coming from agent seems to be higher, let's break it down further.

In [128]:
column_list = ['agent']
city_agent_df = cancelation_ratio(cityHotelDF, column_list)
city_agent_df.sort(col("cancelation_ratio").desc()).where(col("count") > 100).show()

+-----+-----+------------+-----------------+
|agent|count|cancel_count|cancelation_ratio|
+-----+-----+------------+-----------------+
|  326|  160|         160|              1.0|
|    5|  102|         102|              1.0|
|  236|  247|         247|              1.0|
|   31|  162|         154|           0.9506|
|   58|  335|         295|           0.8806|
|   29|  679|         544|           0.8012|
|   44|  292|         232|           0.7945|
|   19| 1061|         780|           0.7352|
|    1| 7137|        5230|           0.7328|
|  154|  187|         135|           0.7219|
|  119|  304|         211|           0.6941|
|   56|  375|         256|           0.6827|
|   20|  540|         359|           0.6648|
|  134|  195|         122|           0.6256|
|  229|  786|         484|           0.6158|
|   34|  293|         179|           0.6109|
|   35|  109|          65|           0.5963|
|    3| 1308|         767|           0.5864|
|   37| 1230|         717|           0.5829|
|   86|  3

In [129]:
column_list = ['agent']
resort_agent_df = cancelation_ratio(resortHotelDF, column_list)
resort_agent_df.sort(col("cancelation_ratio").desc()).where(col("count") > 100).show()

+-----+-----+------------+-----------------+
|agent|count|cancel_count|cancelation_ratio|
+-----+-----+------------+-----------------+
|   68|  211|         165|            0.782|
|   11|  105|          75|           0.7143|
|  248|  131|          83|           0.6336|
|  134|  287|         177|           0.6167|
|   67|  127|          76|           0.5984|
|   38|  236|         132|           0.5593|
|  208|  173|          95|           0.5491|
|   96|  537|         244|           0.4544|
|  315|  256|         102|           0.3984|
|  240|13905|        5483|           0.3943|
|   15|  242|          93|           0.3843|
|  410|  133|          47|           0.3534|
|  273|  349|         121|           0.3467|
|  242|  779|         260|           0.3338|
|  142|  137|          45|           0.3285|
|  298|  472|         142|           0.3008|
|  175|  195|          48|           0.2462|
|  177|  327|          78|           0.2385|
|  147|  133|          31|           0.2331|
|    5|  2

---

In [131]:
column_list = ['hotel','market_segment']
market_segment_df = cancelation_ratio(booking_df, column_list)
market_segment_df.show()

+------------+--------------+-----+------------+-----------------+
|       hotel|market_segment|count|cancel_count|cancelation_ratio|
+------------+--------------+-----+------------+-----------------+
|  City Hotel|      Aviation|  237|          52|           0.2194|
|  City Hotel| Complementary|  542|          64|           0.1181|
|  City Hotel|     Corporate| 2986|         641|           0.2147|
|  City Hotel|        Direct| 6093|        1056|           0.1733|
|  City Hotel|        Groups|13975|        9623|           0.6886|
|  City Hotel| Offline TA/TO|16747|        7173|           0.4283|
|  City Hotel|     Online TA|38748|       14491|            0.374|
|  City Hotel|     Undefined|    2|           2|              1.0|
|Resort Hotel| Complementary|  201|          33|           0.1642|
|Resort Hotel|     Corporate| 2309|         351|            0.152|
|Resort Hotel|        Direct| 6513|         878|           0.1348|
|Resort Hotel|        Groups| 5836|        2474|           0.4

In [130]:
column_list = ['hotel','market_segment', 'distribution_channel']
market_segment_channel_df = cancelation_ratio(booking_df, column_list)
market_segment_channel_df.show(100)

+------------+--------------+--------------------+-----+------------+-----------------+
|       hotel|market_segment|distribution_channel|count|cancel_count|cancelation_ratio|
+------------+--------------+--------------------+-----+------------+-----------------+
|  City Hotel|      Aviation|           Corporate|  227|          51|           0.2247|
|  City Hotel|      Aviation|               TA/TO|   10|           1|              0.1|
|  City Hotel| Complementary|           Corporate|   69|          17|           0.2464|
|  City Hotel| Complementary|              Direct|  437|          46|           0.1053|
|  City Hotel| Complementary|               TA/TO|   36|           1|           0.0278|
|  City Hotel|     Corporate|           Corporate| 2636|         544|           0.2064|
|  City Hotel|     Corporate|              Direct|   56|           3|           0.0536|
|  City Hotel|     Corporate|               TA/TO|  294|          94|           0.3197|
|  City Hotel|        Direct|   

Offline travel agents have lower cancellation rates for the resort hotel, while not so for the city hotel.

The segmentation of Groups is bringing sizable orders for both hotels but also has a higher than average cancelation rate.

---

In [132]:
column_list = ['country']
country_df = cancelation_ratio(booking_df, column_list)
country_df.sort(col("cancelation_ratio").desc()).where(col("count") > 100).show()

+-------+-----+------------+-----------------+
|country|count|cancel_count|cancelation_ratio|
+-------+-----+------------+-----------------+
|    PRT|48590|       27519|           0.5664|
|    AGO|  362|         205|           0.5663|
|    CHN|  999|         462|           0.4625|
|    MAR|  259|         109|           0.4208|
|    KOR|  133|          55|           0.4135|
|    TUR|  248|         102|           0.4113|
|    LUX|  287|         109|           0.3798|
|    RUS|  632|         239|           0.3782|
|    BRA| 2224|         830|           0.3732|
|    ITA| 3766|        1333|            0.354|
|    HUN|  230|          77|           0.3348|
|    NOR|  607|         181|           0.2982|
|    GRC|  128|          35|           0.2734|
|    ROU|  500|         134|            0.268|
|    ESP| 8568|        2177|           0.2541|
|    ISR|  669|         169|           0.2526|
|    ARG|  214|          54|           0.2523|
|    AUS|  426|         107|           0.2512|
|    DNK|  43

In [133]:
column_list = ['country']
city_country_df = cancelation_ratio(cityHotelDF, column_list)
city_country_df.sort(col("cancelation_ratio").desc()).where(col("count") > 100).show()

+-------+-----+------------+-----------------+
|country|count|cancel_count|cancelation_ratio|
+-------+-----+------------+-----------------+
|    PRT|30960|       20081|           0.6486|
|    AGO|  338|         198|           0.5858|
|    CHN|  865|         453|           0.5237|
|    TUR|  225|          95|           0.4222|
|    KOR|  124|          52|           0.4194|
|    BRA| 1794|         729|           0.4064|
|    RUS|  443|         178|           0.4018|
|    LUX|  207|          83|            0.401|
|    MAR|  184|          73|           0.3967|
|    ITA| 3307|        1253|           0.3789|
|    HUN|  183|          68|           0.3716|
|    IRL| 1209|         400|           0.3309|
|    NOR|  484|         158|           0.3264|
|    ROU|  323|         102|           0.3158|
|    POL|  586|         176|           0.3003|
|    GBR| 5315|        1562|           0.2939|
|    ESP| 4611|        1326|           0.2876|
|    ARG|  157|          45|           0.2866|
|    DNK|  37

In [134]:
column_list = ['country']
resort_country_df = cancelation_ratio(resortHotelDF, column_list)
resort_country_df.sort(col("cancelation_ratio").desc()).where(col("count") > 100).show()

+-------+-----+------------+-----------------+
|country|count|cancel_count|cancelation_ratio|
+-------+-----+------------+-----------------+
|    PRT|17630|        7438|           0.4219|
|    RUS|  189|          61|           0.3228|
|    CHE|  435|         112|           0.2575|
|    SWE|  304|          73|           0.2401|
|    BRA|  430|         101|           0.2349|
|    ESP| 3957|         851|           0.2151|
|    IRL| 2166|         432|           0.1994|
|    NOR|  123|          23|            0.187|
|    ROU|  177|          32|           0.1808|
|    ITA|  459|          80|           0.1743|
|    AUT|  210|          34|           0.1619|
|    USA|  479|          72|           0.1503|
|     CN|  710|          96|           0.1352|
|    BEL|  448|          59|           0.1317|
|    FRA| 1611|         211|            0.131|
|    GBR| 6814|         891|           0.1308|
|    DEU| 1203|         146|           0.1214|
|    POL|  333|          39|           0.1171|
|    NLD|  51

---

In [135]:
column_list = ['hotel','meal']
meal_df = cancelation_ratio(booking_df, column_list)
meal_df.show()

+------------+---------+-----+------------+-----------------+
|       hotel|     meal|count|cancel_count|cancelation_ratio|
+------------+---------+-----+------------+-----------------+
|  City Hotel|       BB|62305|       26667|            0.428|
|  City Hotel|       FB|   44|          35|           0.7955|
|  City Hotel|       HB| 6417|        2437|           0.3798|
|  City Hotel|       SC|10564|        3963|           0.3751|
|Resort Hotel|       BB|30005|        7843|           0.2614|
|Resort Hotel|       FB|  754|         443|           0.5875|
|Resort Hotel|       HB| 8046|        2547|           0.3166|
|Resort Hotel|       SC|   86|           3|           0.0349|
|Resort Hotel|Undefined| 1169|         286|           0.2447|
+------------+---------+-----+------------+-----------------+



Guests who've booked full board have been more likely to cancel.

---

In [136]:
column_list = ['hotel','lead_time_cat']
hotel_lead_time_df = cancelation_ratio(booking_df, column_list)
hotel_lead_time_df.show()

+------------+-------------+-----+------------+-----------------+
|       hotel|lead_time_cat|count|cancel_count|cancelation_ratio|
+------------+-------------+-----+------------+-----------------+
|  City Hotel|  1. same_day| 3109|         273|           0.0878|
|  City Hotel|     2. short| 7699|        1046|           0.1359|
|  City Hotel|    3. medium|33351|       12180|           0.3652|
|  City Hotel|      4. long|18223|        8746|           0.4799|
|  City Hotel|5. super_long|16948|       10857|           0.6406|
|Resort Hotel|  1. same_day| 3236|         157|           0.0485|
|Resort Hotel|     2. short| 5702|         426|           0.0747|
|Resort Hotel|    3. medium|15162|        4244|           0.2799|
|Resort Hotel|      4. long| 8216|        3075|           0.3743|
|Resort Hotel|5. super_long| 7744|        3220|           0.4158|
+------------+-------------+-----+------------+-----------------+



The longer the lead time is, the more likely the order is to be canceled, which is very natural. 

However, if the cancelation gap is large enough, there are high chances that the hotel could resell this room.

---

In [137]:
column_list = ['hotel','booking_change_cat']
hotel_booking_changes_df = cancelation_ratio(booking_df, column_list)
hotel_booking_changes_df.show()

+------------+------------------+-----+------------+-----------------+
|       hotel|booking_change_cat|count|cancel_count|cancelation_ratio|
+------------+------------------+-----+------------+-----------------+
|  City Hotel|           1. none|69062|       31423|            0.455|
|  City Hotel|            2. one|  313|          16|           0.0511|
|  City Hotel|           3. some|  626|          35|           0.0559|
|  City Hotel|           4. many| 9329|        1628|           0.1745|
|Resort Hotel|           1. none|32252|        9968|           0.3091|
|Resort Hotel|            2. one|  241|          10|           0.0415|
|Resort Hotel|           3. some|  428|          22|           0.0514|
|Resort Hotel|           4. many| 7139|        1122|           0.1572|
+------------+------------------+-----+------------+-----------------+



We've found out if the guests made just 1 or a few changes to the booking, it has a much lower cancelation rate.

Whereas if the guests made either no or many change(s), they are more likely to cancel. 

We could suggest the hotels to contact the guests beforehand (e.g. 2 weeks before) if there's any changes they'd like to do, maybe those who want to cancel would then cancel, and hence give the hotel more time to resell the room.

In [146]:
column_list = ['guest_size','adr_cat']
city_size_adr_df = cancelation_ratio(cityHotelDF, column_list)
city_size_adr_df.show()

+--------------+----------------+-----+------------+-----------------+
|    guest_size|         adr_cat|count|cancel_count|cancelation_ratio|
+--------------+----------------+-----+------------+-----------------+
|       1. solo|1. below_average| 5456|        1506|            0.276|
|       1. solo|      2. average| 7566|        2770|           0.3661|
|       1. solo|3. above_average| 2546|        1096|           0.4305|
|     2. couple|1. below_average|14387|        7512|           0.5221|
|     2. couple|      2. average|28783|       12216|           0.4244|
|     2. couple|3. above_average|10765|        4491|           0.4172|
|3. small_group|1. below_average|  309|          52|           0.1683|
|3. small_group|      2. average| 2202|         668|           0.3034|
|3. small_group|3. above_average| 7147|        2768|           0.3873|
|  4. big_group|1. below_average|  157|          23|           0.1465|
|  4. big_group|      2. average|    8|           0|              0.0|
|  4. 

In [139]:
column_list = ['guest_size','lead_time_cat','adr_cat']
city_size_adr_df = cancelation_ratio(cityHotelDF, column_list)
city_size_adr_df.show(50)

+--------------+-------------+----------------+-----+------------+-----------------+
|    guest_size|lead_time_cat|         adr_cat|count|cancel_count|cancelation_ratio|
+--------------+-------------+----------------+-----+------------+-----------------+
|       1. solo|  1. same_day|1. below_average|  490|          37|           0.0755|
|       1. solo|  1. same_day|      2. average|  447|          21|            0.047|
|       1. solo|  1. same_day|3. above_average|  197|          19|           0.0964|
|       1. solo|     2. short|1. below_average| 1372|         186|           0.1356|
|       1. solo|     2. short|      2. average| 1273|         148|           0.1163|
|       1. solo|     2. short|3. above_average|  433|          66|           0.1524|
|       1. solo|    3. medium|1. below_average| 2068|         553|           0.2674|
|       1. solo|    3. medium|      2. average| 3253|        1365|           0.4196|
|       1. solo|    3. medium|3. above_average| 1185|         389

In [140]:
column_list = ['guest_size','adr_cat']
resort_size_adr_df = cancelation_ratio(resortHotelDF, column_list)
resort_size_adr_df.show()

+--------------+----------------+-----+------------+-----------------+
|    guest_size|         adr_cat|count|cancel_count|cancelation_ratio|
+--------------+----------------+-----+------------+-----------------+
|       1. solo|1. below_average| 3984|         577|           0.1448|
|       1. solo|      2. average| 2669|         494|           0.1851|
|       1. solo|3. above_average|  360|         112|           0.3111|
|     2. couple|1. below_average| 5480|        1456|           0.2657|
|     2. couple|      2. average|16180|        4747|           0.2934|
|     2. couple|3. above_average| 6456|        2150|            0.333|
|3. small_group|1. below_average|  233|          34|           0.1459|
|3. small_group|      2. average| 1501|         387|           0.2578|
|3. small_group|3. above_average| 3169|        1148|           0.3623|
|  4. big_group|1. below_average|   27|          16|           0.5926|
|  4. big_group|3. above_average|    1|           1|              1.0|
+-----

For the resort hotel, the higher the average daily rate is, the more likely the order will be cancelled. However, for the city hotel in the couple segment, it is not the case. 

The super long lead time combination with a below average has a cancellation for 84%, which is much higher than the baseline (41%)

In [141]:
column_list = ['hotel','deposit_type']
hotel_deposit_df = cancelation_ratio(booking_df, column_list)
hotel_deposit_df.show()

+------------+------------+-----+------------+-----------------+
|       hotel|deposit_type|count|cancel_count|cancelation_ratio|
+------------+------------+-----+------------+-----------------+
|  City Hotel|  No Deposit|66442|       20244|           0.3047|
|  City Hotel|  Non Refund|12868|       12844|           0.9981|
|  City Hotel|  Refundable|   20|          14|              0.7|
|Resort Hotel|  No Deposit|38199|        9450|           0.2474|
|Resort Hotel|  Non Refund| 1719|        1650|           0.9599|
|Resort Hotel|  Refundable|  142|          22|           0.1549|
+------------+------------+-----+------------+-----------------+



In [142]:
column_list = ['hotel','deposit_type','is_agent']
hotel_deposit_df = cancelation_ratio(booking_df, column_list)
hotel_deposit_df.show()

+------------+------------+--------+-----+------------+-----------------+
|       hotel|deposit_type|is_agent|count|cancel_count|cancelation_ratio|
+------------+------------+--------+-----+------------+-----------------+
|  City Hotel|  No Deposit|       0| 6574|        1076|           0.1637|
|  City Hotel|  No Deposit|       1|59868|       19168|           0.3202|
|  City Hotel|  Non Refund|       0| 1554|        1533|           0.9865|
|  City Hotel|  Non Refund|       1|11314|       11311|           0.9997|
|  City Hotel|  Refundable|       0|    3|           0|              0.0|
|  City Hotel|  Refundable|       1|   17|          14|           0.8235|
|Resort Hotel|  No Deposit|       0| 7601|         930|           0.1224|
|Resort Hotel|  No Deposit|       1|30598|        8520|           0.2784|
|Resort Hotel|  Non Refund|       0|  497|         479|           0.9638|
|Resort Hotel|  Non Refund|       1| 1222|        1171|           0.9583|
|Resort Hotel|  Refundable|       0|  

In [143]:
column_list = ['hotel','deposit_type', 'stay_length_cat']
hotel_deposit_df = cancelation_ratio(booking_df, column_list)
hotel_deposit_df.show()

+------------+------------+---------------+-----+------------+-----------------+
|       hotel|deposit_type|stay_length_cat|count|cancel_count|cancelation_ratio|
+------------+------------+---------------+-----+------------+-----------------+
|  City Hotel|  No Deposit|         1. one|12096|        2938|           0.2429|
|  City Hotel|  No Deposit|         2. two|15362|        4376|           0.2849|
|  City Hotel|  No Deposit|       3. three|17368|        5480|           0.3155|
|  City Hotel|  No Deposit|      4. medium|18029|        5994|           0.3325|
|  City Hotel|  No Deposit|        5. long| 3587|        1456|           0.4059|
|  City Hotel|  Non Refund|         1. one| 1173|        1165|           0.9932|
|  City Hotel|  Non Refund|         2. two| 6059|        6054|           0.9992|
|  City Hotel|  Non Refund|       3. three| 4009|        4004|           0.9988|
|  City Hotel|  Non Refund|      4. medium| 1579|        1573|           0.9962|
|  City Hotel|  Non Refund| 

For the non-refundable orders, curiously for both hotels, have a very high cancellation rate (99% for city hotels and 95% for resort hotels).

Further attributes were considered but cannot seem to find a confounding factor. 

---

In [144]:
column_list = ['hotel','season_cat']
hotel_season_df = cancelation_ratio(booking_df, column_list)
hotel_season_df.show()

+------------+--------------+-----+------------+-----------------+
|       hotel|    season_cat|count|cancel_count|cancelation_ratio|
+------------+--------------+-----+------------+-----------------+
|  City Hotel|1. high season|32908|       13829|           0.4202|
|  City Hotel|      2. other|29232|       12489|           0.4272|
|  City Hotel| 3. low season|17190|        6784|           0.3946|
|Resort Hotel|1. high season|16581|        5075|           0.3061|
|Resort Hotel|      2. other|13098|        3835|           0.2928|
|Resort Hotel| 3. low season|10381|        2212|           0.2131|
+------------+--------------+-----+------------+-----------------+



For both hotels, orders arriving in the low season (month of February, November, December and January) have lower cancellation rates, especially more apparent for the resort hotel.

---

In [145]:
column_list = ['hotel','previous_cancellations']
hotel_prev_cancel_df = cancelation_ratio(booking_df, column_list)
hotel_prev_cancel_df.show()

+------------+----------------------+-----+------------+-----------------+
|       hotel|previous_cancellations|count|cancel_count|cancelation_ratio|
+------------+----------------------+-----+------------+-----------------+
|  City Hotel|                     0|73941|       28084|           0.3798|
|  City Hotel|                     1| 5155|        4964|           0.9629|
|  City Hotel|                     2|   72|          13|           0.1806|
|  City Hotel|                     3|   51|           7|           0.1373|
|  City Hotel|                     4|   25|           4|             0.16|
|  City Hotel|                     5|   16|           1|           0.0625|
|  City Hotel|                     6|   22|           7|           0.3182|
|  City Hotel|                    11|   35|          10|           0.2857|
|  City Hotel|                    13|   12|          11|           0.9167|
|  City Hotel|                    21|    1|           1|              1.0|
|Resort Hotel|           

Those who have canceled previously one time are very likely to cancel again for both hotels. 

However for the city hotel, those who have canceled 2 or 3 times are actually less likely to cancel compared to a new guest. 